In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from casestudy.update import auto_update

In [4]:
from decouple import config

In [5]:
import django
django.setup()

In [10]:
from casestudy.update import make_baseframe, auto_update

In [7]:
baseframe = make_baseframe()

initializing
adding fatalities
adding cases
adding tests
adding pollutants
adding measurements
backfill time-static data
adding strindex
adding Google Mobility
adding Apple Mobility
adding Causes of Death
adding Travel popularity
adding GDP
COMPLETE


In [11]:
auto_update()

Running get_italy...
Running get_braz...
Running get_US...
Running create_rest...
Running update_austests...
Running update_cadtests...
Running update_ustests...
Running update_resttests...
Running update_strindex...
Running update_gmobi...


/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


Aruba AW
Cote d'Ivoire 
Cape Verde CV
Laos LA
North Macedonia MK
Réunion RE
Tajikistan TJ
Running update_amobi...
Instantiating Chrome WebDriver...
Running update_msmts...
Exception('no data is available within your requested subset. Request returned no data.')
Running update_pollutants...
TimeoutError('update_pollutants Timed Out')
Running make_baseframe...
initializing
adding fatalities
adding cases
adding tests
adding pollutants
adding measurements
backfill time-static data
adding strindex
adding Google Mobility
adding Apple Mobility
adding Causes of Death
adding Travel popularity
adding GDP
COMPLETE
Running test_region_consistency...
Running test_notnas...
Running test_notnas...
Running test_notnas...
Running test_notnas...
Running test_notnas...
Running test_notnas...
Exception(' ...Failing Regions: [407]')
Running test_duplicate_dates...
Running test_duplicate_days...
Running test_negative_days...
Running make_baseframe...
initializing
adding fatalities
adding cases
adding tests


/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/nbconvert/filters/datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/javascript', 'application/vnd.bokehjs_load.v0+json']) is not able to be represented.
  mimetypes=output.keys())
/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/nbconvert/filters/datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/javascript', 'application/vnd.bokehjs_exec.v0+json']) is not able to be represented.
  mimetypes=output.keys())
